<a href="https://colab.research.google.com/github/Roman3173/RomanGo/blob/main/Test%20cases/3D_room_test/3d_ml_test_case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подготовка данных

In [1]:
import cv2
import numpy as np
import json

In [2]:
# image_1024_aligned_rgb.png
!gdown 1MEa-_oowpOuP2wa7zEmSM7uYxcgNAXxt

Downloading...
From: https://drive.google.com/uc?id=1MEa-_oowpOuP2wa7zEmSM7uYxcgNAXxt
To: /content/image_1024_aligned_rgb.png
100% 571k/571k [00:00<00:00, 121MB/s]


In [3]:
# test_object_lu.json
!gdown 15qyneDxjA30lOdrZOHX80FrQVWSKIpnJ

Downloading...
From: https://drive.google.com/uc?id=15qyneDxjA30lOdrZOHX80FrQVWSKIpnJ
To: /content/test_object_lu.json
100% 194/194 [00:00<00:00, 641kB/s]


In [4]:
# floor_plan_test.jpg
!gdown 1vlmfA20KIyGlHmhIoSzus4NFIAuoRFb9

Downloading...
From: https://drive.google.com/uc?id=1vlmfA20KIyGlHmhIoSzus4NFIAuoRFb9
To: /content/floor_plan_test.jpg
100% 17.8k/17.8k [00:00<00:00, 43.7MB/s]


In [5]:
# v_corners.npy
!gdown 18cKH92wswG-_Hk-uZYqJmYeem6GMOafb

Downloading...
From: https://drive.google.com/uc?id=18cKH92wswG-_Hk-uZYqJmYeem6GMOafb
To: /content/v_corners.npy
100% 224/224 [00:00<00:00, 1.12MB/s]


In [6]:
# h_corners.npy
!gdown 1VPSbwA8Y8roX7-nMnBuwtKRCGihRLhAY

Downloading...
From: https://drive.google.com/uc?id=1VPSbwA8Y8roX7-nMnBuwtKRCGihRLhAY
To: /content/h_corners.npy
100% 8.32k/8.32k [00:00<00:00, 22.9MB/s]


In [7]:
# /content/depth_map.npy
!gdown 1x6YD2uk-ivS9uhosIkY75-k2hDVIhte6

Downloading...
From: https://drive.google.com/uc?id=1x6YD2uk-ivS9uhosIkY75-k2hDVIhte6
To: /content/depth_map.npy
100% 2.10M/2.10M [00:00<00:00, 24.0MB/s]


In [8]:
panorama_image_path = "/content/image_1024_aligned_rgb.png"
panorama_image = cv2.imread(panorama_image_path, cv2.IMREAD_COLOR)

In [9]:
floor_plan_path = "/content/floor_plan_test.jpg"
floor_plan = cv2.imread(panorama_image_path)

In [10]:
json_path = "/content/test_object_lu.json"
with open(json_path, 'r') as json_file:
    walls_data = json.load(json_file)
# walls_data

In [11]:
v_corners_path = "/content/v_corners.npy"
vertical_corners = np.load(v_corners_path)
# vertical_corners

In [12]:
h_corners_path = "/content/h_corners.npy"
horizontal_corners = np.load(h_corners_path)
# horizontal_corners

In [13]:
depth_map_path = "/content/depth_map.npy"
depth_map = np.load(depth_map_path)
#depth_map

In [14]:
vertical_corners = np.around(vertical_corners, decimals = 0).astype(int)

In [15]:
horizontal_corners = np.around(horizontal_corners, decimals = 0).astype(int)

# 1 стена

In [16]:
panorama_image_path = "/content/image_1024_aligned_rgb.png"
result = cv2.imread(panorama_image_path, cv2.IMREAD_COLOR)
x_min = vertical_corners[0][0]
x_max = vertical_corners[2][0]
y1_max = 512
y2_max = 0
for i in range(x_min, x_max + 1):
    y1_max = min(y1_max, horizontal_corners[0][i])
    y2_max = max(y2_max, horizontal_corners[1][i])

In [17]:
width = 2
for j in range(x_min, x_max + 1, width):
    x1 = j
    x2 = j + width - 1

    # от верхней грани до высоты камеры
    yy = 255
    point1 = [x1, horizontal_corners[0][x1]]
    point2 = [x2, horizontal_corners[0][x2]]
    point3 = [x2, yy]
    point4 = [x1, yy]

    height = yy - y1_max + 1
    src_points = np.array([point1, point2, point3, point4], dtype=np.float32)
    dst_points = np.array([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]], dtype=np.float32)

    # матрица преобразования
    perspective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)

    # преобразование
    result_rectangular = cv2.warpPerspective(result, perspective_matrix, (width, height))

    for i in range(len(result_rectangular)):
        for j in range(width):
            result[i + y1_max][x1 + j] = result_rectangular[i][j]



    # от высоты камеры до нижней грани
    yy = 256
    point1 = [x1, yy]
    point2 = [x2, yy]
    point3 = [x2, horizontal_corners[1][x2]]
    point4 = [x1, horizontal_corners[1][x1]]

    height = y2_max - yy + 1
    src_points = np.array([point1, point2, point3, point4], dtype=np.float32)
    dst_points = np.array([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]], dtype=np.float32)

    # Получите матрицу преобразования
    perspective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)

    # Примените преобразование
    result_rectangular = cv2.warpPerspective(result, perspective_matrix, (width, height))

    for i in range(len(result_rectangular)):
        for k in range(width):
            result[i + yy][x1 + k] = result_rectangular[i][k]

In [18]:
# обрезаем стену по граням.
result = result[y1_max:y2_max, x_min:x_max]
#  # сохранить промежуточный результат
# cv2.imwrite("/content/1_wall.jpg", result)

In [19]:
depth_1_wall = depth_map[255][31:363]
center_1_wall = 255-30
mera = (depth_1_wall[center_1_wall + 3] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5

In [20]:
ww =   ((depth_1_wall[0] **2 - depth_1_wall[center_1_wall]**2) ** 0.5
      + (depth_1_wall[-1]**2 - depth_1_wall[center_1_wall]**2) ** 0.5)
ww / 6.04

1.099602512686297

In [21]:
total_size = 0 + 4
ind_size_higher = [[center_1_wall, 4]]
ind = center_1_wall + 3
prev = (depth_1_wall[center_1_wall + 3] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5
while ind + 4 < len(depth_1_wall):
    cur = (depth_1_wall[ind + 4] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5
    next_size = np.around((cur - prev) / mera * 4, decimals = 0).astype(int)
    ind_size_higher.append([ind, next_size])
    total_size += next_size
    ind += 4
    prev = cur
if ind != depth_1_wall.shape[0] - 1:
    ind -= 4
    total_size -= ind_size_higher[-1][1]
    ind_size_higher.pop()
    step = depth_1_wall.shape[0] - 1 - ind
    cur =  ((depth_1_wall[ind + step] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5)
    prev = ((depth_1_wall[ind] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5)
    next_size = np.around((cur - prev) / mera * 4, decimals = 0).astype(int)
    ind_size_higher.append([ind, next_size])
    total_size += next_size
    ind += step
ind_size_higher.append([ind, 0])
high_size = total_size
#ind_size_higher

In [22]:
total_size = 4
ind = center_1_wall - 4
ind_size_lower = [[ind, 4]]
ind -= 4
prev = (depth_1_wall[center_1_wall + 3] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5
while ind >= 0:
    cur = ((depth_1_wall[ind] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5)
    next_size = np.around((cur - prev) / mera * 4, decimals = 0).astype(int)
    ind_size_lower.insert(0, [ind, next_size])
    total_size += next_size
    ind -= 4
    prev = cur
if ind != 0:
    ind += 4 * 2
    total_size -= ind_size_lower[0][1]
    ind_size_lower.pop(0)
    step = ind
    prev = ((depth_1_wall[ind] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5)
    cur = ((depth_1_wall[0] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5)
    next_size = np.around((cur - prev)/ mera * 4, decimals = 0).astype(int)
    ind_size_lower.insert(0, [0, next_size])
    total_size += next_size
    ind = 0
total_size += high_size
#ind_size_lower

In [23]:
ind_size = ind_size_lower + ind_size_higher
new_res = np.zeros((result.shape[0], total_size, result.shape[2]), dtype=np.uint8)
#new_res.shape

In [24]:
height = result.shape[0]
xx = 0
for j in range(len(ind_size) - 1):
    width = ind_size[j + 1][0] - ind_size[j][0]
    x1 = ind_size[j][0]
    x2 = x1 + width - 1

    yy = 255
    point1 = [x1, 0]
    point2 = [x2, 0]
    point3 = [x2, height - 1]
    point4 = [x1, height - 1]


    src_points = np.array([point1, point2, point3, point4], dtype=np.float32)
    dst_points = np.array([[0, 0], [ind_size[j][1] - 1, 0], [ind_size[j][1] - 1, height - 1], [0, height - 1]], dtype=np.float32)

    # Получите матрицу преобразования
    perspective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)

    # Примените преобразование
    result_rectangular = cv2.warpPerspective(result, perspective_matrix, (ind_size[j][1], height))

    for i in range(len(result_rectangular)):
        for k in range(ind_size[j][1]):
            new_res[i][xx + k] = result_rectangular[i][k]
    xx += ind_size[j][1]

In [25]:
new_width = 602
new_height = 250

# Сжимаем изображение до нового размера
resized_image = cv2.resize(new_res, (new_width, new_height))
# получаем итоговое изображение с поправкой на соотношение сторон 6020x2500 мм
cv2.imwrite("/content/0_proj.jpg", resized_image)

True

# 2 стена

In [26]:
panorama_image_path = "/content/image_1024_aligned_rgb.png"
result = cv2.imread(panorama_image_path, cv2.IMREAD_COLOR)
x_min = vertical_corners[2][0]
x_max = vertical_corners[4][0]
y1_max = 512
y2_max = 0
for i in range(x_min, x_max + 1):
    y1_max = min(y1_max, horizontal_corners[0][i])
    y2_max = max(y2_max, horizontal_corners[1][i])

In [27]:
width = 2
for j in range(x_min, x_max + 1, width):
    x1 = j
    x2 = j + width - 1

    # от верхней грани до высоты камеры
    yy = 255
    point1 = [x1, horizontal_corners[0][x1]]
    point2 = [x2, horizontal_corners[0][x2]]
    point3 = [x2, yy]
    point4 = [x1, yy]

    height = yy - y1_max + 1
    src_points = np.array([point1, point2, point3, point4], dtype=np.float32)
    dst_points = np.array([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]], dtype=np.float32)

    # матрица преобразования
    perspective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)

    # преобразование
    result_rectangular = cv2.warpPerspective(result, perspective_matrix, (width, height))

    for i in range(len(result_rectangular)):
        for j in range(width):
            result[i + y1_max][x1 + j] = result_rectangular[i][j]



    # от высоты камеры до нижней грани
    yy = 256
    point1 = [x1, yy]
    point2 = [x2, yy]
    point3 = [x2, horizontal_corners[1][x2]]
    point4 = [x1, horizontal_corners[1][x1]]

    height = y2_max - yy + 1
    src_points = np.array([point1, point2, point3, point4], dtype=np.float32)
    dst_points = np.array([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]], dtype=np.float32)

    # Получите матрицу преобразования
    perspective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)

    # Примените преобразование
    result_rectangular = cv2.warpPerspective(result, perspective_matrix, (width, height))

    for i in range(len(result_rectangular)):
        for k in range(width):
            result[i + yy][x1 + k] = result_rectangular[i][k]

In [28]:
result = result[y1_max:y2_max, x_min:x_max]
#  # сохранить промежуточный результат
# cv2.imwrite("/content/2_wall.jpg", result)

In [29]:
# Нормаль к 2 стене
depth_1_wall = depth_map[255][x_min:x_max]
depth_1_wall[148:150] - min(depth_1_wall)

array([0., 0.], dtype=float32)

In [30]:
center_1_wall = 149
mera = (depth_1_wall[center_1_wall + 3] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5

In [31]:
mera
(depth_1_wall[center_1_wall - 4] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5 - mera

0.0

In [32]:
ww =   ((depth_1_wall[0] **2 - depth_1_wall[center_1_wall]**2) ** 0.5
      + (depth_1_wall[-1]**2 - depth_1_wall[center_1_wall]**2) ** 0.5)
ww / 2.05

1.196375528015284

In [33]:
wr1 = (depth_1_wall[-1] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5
(wr1 / ww * 2.05)

1.1061898785993762

In [34]:
x_min = vertical_corners[2][0]
x_max = vertical_corners[4][0]
depth_1_wall = depth_map[255][x_min:x_max]
depth_1_wall[148:150] - min(depth_1_wall)
center_1_wall = 149
HH = (2 * depth_1_wall[0]**2 + 2 * depth_1_wall[-1]**2 - 4 * depth_1_wall[center_1_wall] ** 2) ** 0.5
HH / 2.05

1.200122789218613

In [35]:
total_size = 0 + 4
ind_size_higher = [[center_1_wall, 4]]
ind = center_1_wall + 3
prev = (depth_1_wall[center_1_wall + 3] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5
while ind + 4 < len(depth_1_wall):
    cur = (depth_1_wall[ind + 4] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5
    next_size = np.around((cur - prev) / mera * 4, decimals = 0).astype(int)
    ind_size_higher.append([ind, next_size])
    total_size += next_size
    ind += 4
    prev = cur
if ind != depth_1_wall.shape[0] - 1:
    ind -= 4
    total_size -= ind_size_higher[-1][1]
    ind_size_higher.pop()
    step = depth_1_wall.shape[0] - 1 - ind
    cur =  ((depth_1_wall[ind + step] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5)
    prev = ((depth_1_wall[ind] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5)
    next_size = np.around((cur - prev) / mera * 4, decimals = 0).astype(int)
    ind_size_higher.append([ind, next_size])
    total_size += next_size
    ind += step
ind_size_higher.append([ind, 0])
high_size = total_size
#ind_size_higher

In [36]:
total_size = 4
ind = center_1_wall - 4
ind_size_lower = [[ind, 4]]
ind -= 4
prev = (depth_1_wall[center_1_wall + 3] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5
while ind >= 0:
    cur = ((depth_1_wall[ind] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5)
    next_size = np.around((cur - prev) / mera * 4, decimals = 0).astype(int)
    ind_size_lower.insert(0, [ind, next_size])
    total_size += next_size
    ind -= 4
    prev = cur
if ind != 0:
    ind += 4 * 2
    total_size -= ind_size_lower[0][1]
    ind_size_lower.pop(0)
    step = ind
    prev = ((depth_1_wall[ind] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5)
    cur = ((depth_1_wall[0] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5)
    next_size = np.around((cur - prev)/ mera * 4, decimals = 0).astype(int)
    ind_size_lower.insert(0, [0, next_size])
    total_size += next_size
    ind = 0
total_size += high_size
#ind_size_lower

In [37]:
ind_size = ind_size_lower + ind_size_higher
new_res = np.zeros((result.shape[0], total_size, result.shape[2]), dtype=np.uint8)
new_res.shape

(329, 529, 3)

In [38]:
height = result.shape[0]
xx = 0
for j in range(len(ind_size) - 1):
    width = ind_size[j + 1][0] - ind_size[j][0]
    x1 = ind_size[j][0]
    x2 = x1 + width - 1

    point1 = [x1, 0]
    point2 = [x2, 0]
    point3 = [x2, height - 1]
    point4 = [x1, height - 1]


    src_points = np.array([point1, point2, point3, point4], dtype=np.float32)
    dst_points = np.array([[0, 0], [ind_size[j][1] - 1, 0], [ind_size[j][1] - 1, height - 1], [0, height - 1]], dtype=np.float32)

    # Получите матрицу преобразования
    perspective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)

    # Примените преобразование
    result_rectangular = cv2.warpPerspective(result, perspective_matrix, (ind_size[j][1], height))

    for i in range(len(result_rectangular)):
        for k in range(ind_size[j][1]):
            new_res[i][xx + k] = result_rectangular[i][k]
    xx += ind_size[j][1]

In [39]:
new_width = 205
new_height = 250

# Сжимаем изображение до нового размера
resized_image = cv2.resize(new_res, (new_width, new_height))
# получаем итоговое изображение с поправкой на соотношение сторон 2050x2500 мм
cv2.imwrite("/content/1_proj.jpg", resized_image)

True

# 3 стена

In [40]:
panorama_image_path = "/content/image_1024_aligned_rgb.png"
result = cv2.imread(panorama_image_path, cv2.IMREAD_COLOR)
x_min = vertical_corners[4][0]
x_max = vertical_corners[6][0]
y1_max = 512
y2_max = 0
for i in range(x_min, x_max + 1):
    y1_max = min(y1_max, horizontal_corners[0][i])
    y2_max = max(y2_max, horizontal_corners[1][i])

In [41]:
width = 2
for j in range(x_min, x_max + 1, width):
    x1 = j
    x2 = j + width - 1

    # от верхней грани до высоты камеры
    yy = 255
    point1 = [x1, horizontal_corners[0][x1]]
    point2 = [x2, horizontal_corners[0][x2]]
    point3 = [x2, yy]
    point4 = [x1, yy]

    height = yy - y1_max + 1
    src_points = np.array([point1, point2, point3, point4], dtype=np.float32)
    dst_points = np.array([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]], dtype=np.float32)

    # матрица преобразования
    perspective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)

    # преобразование
    result_rectangular = cv2.warpPerspective(result, perspective_matrix, (width, height))

    for i in range(len(result_rectangular)):
        for j in range(width):
            result[i + y1_max][x1 + j] = result_rectangular[i][j]



    # от высоты камеры до нижней грани
    yy = 256
    point1 = [x1, yy]
    point2 = [x2, yy]
    point3 = [x2, horizontal_corners[1][x2]]
    point4 = [x1, horizontal_corners[1][x1]]

    height = y2_max - yy + 1
    src_points = np.array([point1, point2, point3, point4], dtype=np.float32)
    dst_points = np.array([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]], dtype=np.float32)

    # Получите матрицу преобразования
    perspective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)

    # Примените преобразование
    result_rectangular = cv2.warpPerspective(result, perspective_matrix, (width, height))

    for i in range(len(result_rectangular)):
        for k in range(width):
            result[i + yy][x1 + k] = result_rectangular[i][k]

In [42]:
result = result[y1_max:y2_max, x_min:x_max]
#  # сохранить промежуточный результат
#cv2.imwrite("/content/3_wall.jpg", result)

In [43]:
# Нормаль к 2 стене
depth_1_wall = depth_map[255][x_min:x_max]
depth_1_wall[94:96] - min(depth_1_wall)

array([0., 0.], dtype=float32)

In [44]:
center_1_wall = 95
mera = (depth_1_wall[center_1_wall + 3] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5

In [45]:
mera
(depth_1_wall[center_1_wall - 4] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5 - mera

0.0

In [46]:
ww =   ((depth_1_wall[0] **2 - depth_1_wall[center_1_wall]**2) ** 0.5
      + (depth_1_wall[-1]**2 - depth_1_wall[center_1_wall]**2) ** 0.5)
ww / 1.64

0.7040685161631258

In [47]:
wr1 = (depth_1_wall[-1] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5
wr1 / (ww / 1.64)

0.3959861237427783

In [48]:
#Расстояние от 3 стены до камеры примерно равно тому, что подсчитано на 2-ой стене, но всё равно не подошло к 4-ой
depth_1_wall[center_1_wall]/ 1.2

1.114448606967926

In [49]:
total_size = 0 + 4
ind_size_higher = [[center_1_wall, 4]]
ind = center_1_wall + 3
prev = (depth_1_wall[center_1_wall + 3] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5
while ind + 4 < len(depth_1_wall):
    cur = (depth_1_wall[ind + 4] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5
    next_size = np.around((cur - prev) / mera * 4, decimals = 0).astype(int)
    ind_size_higher.append([ind, next_size])
    total_size += next_size
    ind += 4
    prev = cur
if ind != depth_1_wall.shape[0] - 1:
    ind -= 4
    total_size -= ind_size_higher[-1][1]
    ind_size_higher.pop()
    step = depth_1_wall.shape[0] - 1 - ind
    cur =  ((depth_1_wall[ind + step] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5)
    prev = ((depth_1_wall[ind] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5)
    next_size = np.around((cur - prev) / mera * 4, decimals = 0).astype(int)
    ind_size_higher.append([ind, next_size])
    total_size += next_size
    ind += step
ind_size_higher.append([ind, 0])
high_size = total_size
#ind_size_higher

In [50]:
total_size = 4
ind = center_1_wall - 4
ind_size_lower = [[ind, 4]]
ind -= 4
prev = (depth_1_wall[center_1_wall + 3] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5
while ind >= 0:
    cur = ((depth_1_wall[ind] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5)
    next_size = np.around((cur - prev) / mera * 4, decimals = 0).astype(int)
    ind_size_lower.insert(0, [ind, next_size])
    total_size += next_size
    ind -= 4
    prev = cur
if ind != 0:
    ind += 4 * 2
    total_size -= ind_size_lower[0][1]
    ind_size_lower.pop(0)
    step = ind
    prev = ((depth_1_wall[ind] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5)
    cur = ((depth_1_wall[0] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5)
    next_size = np.around((cur - prev)/ mera * 4, decimals = 0).astype(int)
    ind_size_lower.insert(0, [0, next_size])
    total_size += next_size
    ind = 0
total_size += high_size
#ind_size_lower

In [51]:
ind_size = ind_size_lower + ind_size_higher
new_res = np.zeros((result.shape[0], total_size, result.shape[2]), dtype=np.uint8)
new_res.shape

(275, 166, 3)

In [52]:
height = result.shape[0]
xx = 0
for j in range(len(ind_size) - 1):
    width = ind_size[j + 1][0] - ind_size[j][0]
    x1 = ind_size[j][0]
    x2 = x1 + width - 1

    point1 = [x1, 0]
    point2 = [x2, 0]
    point3 = [x2, height - 1]
    point4 = [x1, height - 1]


    src_points = np.array([point1, point2, point3, point4], dtype=np.float32)
    dst_points = np.array([[0, 0], [ind_size[j][1] - 1, 0], [ind_size[j][1] - 1, height - 1], [0, height - 1]], dtype=np.float32)

    # Получите матрицу преобразования
    perspective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)

    # Примените преобразование
    result_rectangular = cv2.warpPerspective(result, perspective_matrix, (ind_size[j][1], height))

    for i in range(len(result_rectangular)):
        for k in range(ind_size[j][1]):
            new_res[i][xx + k] = result_rectangular[i][k]
    xx += ind_size[j][1]

In [53]:
new_width = 164
new_height = 250

# Сжимаем изображение до нового размера
resized_image = cv2.resize(new_res, (new_width, new_height))
# получаем итоговое изображение с поправкой на соотношение сторон 1640x2500 мм
cv2.imwrite("/content/2_proj.jpg", resized_image)

True

# 4 стена

In [54]:
panorama_image_path = "/content/image_1024_aligned_rgb.png"
result = cv2.imread(panorama_image_path, cv2.IMREAD_COLOR)
x_min = vertical_corners[6][0]
x_max = vertical_corners[8][0]
y1_max = 512
y2_max = 0
for i in range(x_min, x_max + 1):
    y1_max = min(y1_max, horizontal_corners[0][i])
    y2_max = max(y2_max, horizontal_corners[1][i])

In [55]:
width = 2
for j in range(x_min, x_max + 1, width):
    x1 = j
    x2 = j + width - 1

    # от верхней грани до высоты камеры
    yy = 255
    point1 = [x1, horizontal_corners[0][x1]]
    point2 = [x2, horizontal_corners[0][x2]]
    point3 = [x2, yy]
    point4 = [x1, yy]

    height = yy - y1_max + 1
    src_points = np.array([point1, point2, point3, point4], dtype=np.float32)
    dst_points = np.array([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]], dtype=np.float32)

    # матрица преобразования
    perspective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)

    # преобразование
    result_rectangular = cv2.warpPerspective(result, perspective_matrix, (width, height))

    for i in range(len(result_rectangular)):
        for j in range(width):
            result[i + y1_max][x1 + j] = result_rectangular[i][j]



    # от высоты камеры до нижней грани
    yy = 256
    point1 = [x1, yy]
    point2 = [x2, yy]
    point3 = [x2, horizontal_corners[1][x2]]
    point4 = [x1, horizontal_corners[1][x1]]

    height = y2_max - yy + 1
    src_points = np.array([point1, point2, point3, point4], dtype=np.float32)
    dst_points = np.array([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]], dtype=np.float32)

    # Получите матрицу преобразования
    perspective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)

    # Примените преобразование
    result_rectangular = cv2.warpPerspective(result, perspective_matrix, (width, height))

    for i in range(len(result_rectangular)):
        for k in range(width):
            result[i + yy][x1 + k] = result_rectangular[i][k]

In [56]:
result = result[y1_max:y2_max, x_min:x_max]
#  # сохранить промежуточный результат
#cv2.imwrite("/content/4_wall.jpg", result)

In [57]:
# Нормаль к 2 стене
depth_1_wall = depth_map[255][x_min:x_max]
depth_1_wall

array([1.3511024 , 1.3136218 , 1.2782112 , 1.2447054 , 1.2129558 ,
       1.182829  , 1.154205  , 1.1269749 , 1.1010406 , 1.0763128 ,
       1.0527099 , 1.0301579 , 1.0085893 , 0.98794156, 0.96815807,
       0.9491863 , 0.93097824, 0.91348934, 0.8966785 , 0.88050777,
       0.86494195, 0.84994835, 0.83549666, 0.8215586 , 0.8081079 ,
       0.79511994, 0.7825719 , 0.7704423 , 0.7587111 , 0.7473595 ,
       0.7363699 , 0.72572577, 0.7154115 , 0.70541245, 0.69571495,
       0.6863059 , 0.67717314, 0.66830504, 0.6596908 , 0.6513199 ,
       0.6431828 , 0.63527006, 0.627573  ], dtype=float32)

In [58]:
OA = depth_1_wall[0]
OB = depth_1_wall[-1]
AN = ((OA**2 - OB**2) / (1 - 0.56**2))** 0.5
AB = AN * 0.44
valid_ = AN <= OA
#BN = (OA*OA - AN*AN) ** 0.5
[OA, OB, AN, AB, {'valid?' : valid_}]

[1.3511024,
 0.627573,
 1.4441978999451555,
 0.6354470759758684,
 {'valid?': False}]

In [59]:
center_1_wall = len(depth_1_wall) - 1
mera = (depth_1_wall[center_1_wall - 4] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5 /4
mera

0.05083309764006692

In [60]:
total_size = 4
ind = center_1_wall - 4
ind_size_lower = [[ind, 4]]
ind -= 4
prev = mera
while ind >= 0:
    cur = ((depth_1_wall[ind] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5)
    next_size = np.around((cur - prev) / mera * 4, decimals = 0).astype(int)
    ind_size_lower.insert(0, [ind, next_size])
    total_size += next_size
    ind -= 4
    prev = cur
if ind != 0:
    ind += 4 * 2
    total_size -= ind_size_lower[0][1]
    ind_size_lower.pop(0)
    step = ind
    prev = ((depth_1_wall[ind] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5)
    cur = ((depth_1_wall[0] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5)
    next_size = np.around((cur - prev)/ mera * 4, decimals = 0).astype(int)
    ind_size_lower.insert(0, [0, next_size])
    total_size += next_size
    ind = 0
ind_size_lower.append([center_1_wall, 0])
total_size += - ind_size_lower[-3][1] + 5
ind_size_lower[-3][1] = 5
total_size += - ind_size_lower[-4][1] + 6
ind_size_lower[-4][1] = 6
ind_size_lower

[[0, 18],
 [6, 10],
 [10, 8],
 [14, 8],
 [18, 7],
 [22, 7],
 [26, 6],
 [30, 6],
 [34, 5],
 [38, 4],
 [42, 0]]

In [61]:
ind_size = ind_size_lower
new_res = np.zeros((result.shape[0], total_size, result.shape[2]), dtype=np.uint8)
new_res.shape

(369, 79, 3)

In [62]:
height = result.shape[0]
xx = 0
for j in range(len(ind_size) - 1):
    width = ind_size[j + 1][0] - ind_size[j][0]
    x1 = ind_size[j][0]
    x2 = x1 + width - 1

    point1 = [x1, 0]
    point2 = [x2, 0]
    point3 = [x2, height - 1]
    point4 = [x1, height - 1]


    src_points = np.array([point1, point2, point3, point4], dtype=np.float32)
    dst_points = np.array([[0, 0], [ind_size[j][1] - 1, 0], [ind_size[j][1] - 1, height - 1], [0, height - 1]], dtype=np.float32)

    # Получите матрицу преобразования
    perspective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)

    # Примените преобразование
    result_rectangular = cv2.warpPerspective(result, perspective_matrix, (ind_size[j][1], height))

    for i in range(len(result_rectangular)):
        for k in range(ind_size[j][1]):
            new_res[i][xx + k] = result_rectangular[i][k]
    xx += ind_size[j][1]

In [63]:
new_width = 40
new_height = 250

# Сжимаем изображение до нового размера
resized_image = cv2.resize(result, (new_width, new_height))
# получаем итоговое изображение с поправкой на соотношение сторон 400x2500 мм
cv2.imwrite("/content/3_proj.jpg", resized_image)

True

# 5 стена

In [64]:
panorama_image_path = "/content/image_1024_aligned_rgb.png"
result = cv2.imread(panorama_image_path, cv2.IMREAD_COLOR)
x_min = vertical_corners[8][0]
x_max = vertical_corners[10][0]
y1_max = 512
y2_max = 0
for i in range(x_min, x_max + 1):
    y1_max = min(y1_max, horizontal_corners[0][i])
    y2_max = max(y2_max, horizontal_corners[1][i])

In [65]:
width = 2
for j in range(x_min, x_max + 1, width):
    x1 = j
    x2 = j + width - 1

    # от верхней грани до высоты камеры
    yy = 255
    point1 = [x1, horizontal_corners[0][x1]]
    point2 = [x2, horizontal_corners[0][x2]]
    point3 = [x2, yy]
    point4 = [x1, yy]

    height = yy - y1_max + 1
    src_points = np.array([point1, point2, point3, point4], dtype=np.float32)
    dst_points = np.array([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]], dtype=np.float32)

    # матрица преобразования
    perspective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)

    # преобразование
    result_rectangular = cv2.warpPerspective(result, perspective_matrix, (width, height))

    for i in range(len(result_rectangular)):
        for j in range(width):
            result[i + y1_max][x1 + j] = result_rectangular[i][j]



    # от высоты камеры до нижней грани
    yy = 256
    point1 = [x1, yy]
    point2 = [x2, yy]
    point3 = [x2, horizontal_corners[1][x2]]
    point4 = [x1, horizontal_corners[1][x1]]

    height = y2_max - yy + 1
    src_points = np.array([point1, point2, point3, point4], dtype=np.float32)
    dst_points = np.array([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]], dtype=np.float32)

    # Получите матрицу преобразования
    perspective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)

    # Примените преобразование
    result_rectangular = cv2.warpPerspective(result, perspective_matrix, (width, height))

    for i in range(len(result_rectangular)):
        for k in range(width):
            result[i + yy][x1 + k] = result_rectangular[i][k]

In [66]:
result = result[y1_max:y2_max, x_min:x_max]
#  # сохранить промежуточный результат
#cv2.imwrite("/content/5_wall.jpg", result)

In [67]:
# Нормаль к 2 стене
depth_1_wall = depth_map[255][x_min:x_max]
#depth_1_wall

In [68]:
center_1_wall = 0
mera = (depth_1_wall[center_1_wall + 3] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5
mera

0.05291626340466949

In [69]:
mera = (depth_1_wall[center_1_wall + 7] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5 - mera

In [70]:
total_size = 0 + 4
ind_size_higher = [[center_1_wall, 4]]
ind = center_1_wall + 3
prev = mera
while ind + 4 < len(depth_1_wall):
    cur = (depth_1_wall[ind + 4] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5
    next_size = np.around((cur - prev) / mera * 4, decimals = 0).astype(int)
    ind_size_higher.append([ind, next_size])
    total_size += next_size
    ind += 4
    prev = cur
if ind != depth_1_wall.shape[0] - 1:
    ind -= 4
    total_size -= ind_size_higher[-1][1]
    ind_size_higher.pop()
    step = depth_1_wall.shape[0] - 1 - ind
    cur =  ((depth_1_wall[ind + step] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5)
    prev = ((depth_1_wall[ind] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5)
    next_size = np.around((cur - prev) / mera * 4, decimals = 0).astype(int)
    ind_size_higher.append([ind, next_size])
    total_size += next_size
    ind += step
ind_size_higher.append([ind, 0])
for i in range(3):
    ind_size_higher[i][1] = 2
total_size -= 4
ind_size_higher

[[0, 2],
 [3, 2],
 [7, 2],
 [11, 2],
 [15, 2],
 [19, 2],
 [23, 2],
 [27, 2],
 [31, 2],
 [35, 2],
 [39, 2],
 [43, 2],
 [47, 2],
 [51, 2],
 [55, 2],
 [59, 2],
 [63, 2],
 [67, 2],
 [71, 3],
 [75, 3],
 [79, 3],
 [83, 3],
 [87, 3],
 [91, 4],
 [95, 4],
 [99, 4],
 [103, 5],
 [107, 5],
 [111, 6],
 [115, 6],
 [119, 7],
 [123, 8],
 [127, 10],
 [131, 11],
 [135, 14],
 [139, 17],
 [143, 21],
 [147, 27],
 [151, 35],
 [155, 103],
 [162, 0]]

In [71]:
ind_size = ind_size_higher
new_res = np.zeros((result.shape[0], total_size, result.shape[2]), dtype=np.uint8)
new_res.shape

(373, 338, 3)

In [72]:
height = result.shape[0]
xx = 0
for j in range(len(ind_size) - 1):
    width = ind_size[j + 1][0] - ind_size[j][0]
    x1 = ind_size[j][0]
    x2 = x1 + width - 1

    point1 = [x1, 0]
    point2 = [x2, 0]
    point3 = [x2, height - 1]
    point4 = [x1, height - 1]


    src_points = np.array([point1, point2, point3, point4], dtype=np.float32)
    dst_points = np.array([[0, 0], [ind_size[j][1] - 1, 0], [ind_size[j][1] - 1, height - 1], [0, height - 1]], dtype=np.float32)

    # Получите матрицу преобразования
    perspective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)

    # Примените преобразование
    result_rectangular = cv2.warpPerspective(result, perspective_matrix, (ind_size[j][1], height))

    for i in range(len(result_rectangular)):
        for k in range(ind_size[j][1]):
            new_res[i][xx + k] = result_rectangular[i][k]
    xx += ind_size[j][1]

In [73]:
new_width = 438
new_height = 250

# Сжимаем изображение до нового размера
resized_image = cv2.resize(new_res, (new_width, new_height))
# получаем итоговое изображение с поправкой на соотношение сторон 6020x250 мм
cv2.imwrite("/content/4_proj.jpg", resized_image)

True

# 6 стена

In [74]:
panorama_image_path = "/content/image_1024_aligned_rgb.png"
result = cv2.imread(panorama_image_path, cv2.IMREAD_COLOR)

x_min_1 = vertical_corners[10][0]
x_max_1 = 1023

x_min_2 = 0
x_max_2 = vertical_corners[0][0]

y1_max = 512
y2_max = 0

for i in range(x_min_1, x_max_1 + 1):
    y1_max = min(y1_max, horizontal_corners[0][i])
    y2_max = max(y2_max, horizontal_corners[1][i])

for i in range(x_min_2, x_max_2 + 1):
    y1_max = min(y1_max, horizontal_corners[0][i])
    y2_max = max(y2_max, horizontal_corners[1][i])

[y1_max, y2_max]

[220, 303]

In [75]:
width = 2
for j in range(x_min_1, x_max_1 + 1, width):
    x1 = j
    x2 = j + width - 1

    # от верхней грани до высоты камеры
    yy = 255
    point1 = [x1, horizontal_corners[0][x1]]
    point2 = [x2, horizontal_corners[0][x2]]
    point3 = [x2, yy]
    point4 = [x1, yy]

    height = yy - y1_max + 1
    src_points = np.array([point1, point2, point3, point4], dtype=np.float32)
    dst_points = np.array([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]], dtype=np.float32)

    # матрица преобразования
    perspective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)

    # преобразование
    result_rectangular = cv2.warpPerspective(result, perspective_matrix, (width, height))

    for i in range(len(result_rectangular)):
        for j in range(width):
            result[i + y1_max][x1 + j] = result_rectangular[i][j]



    # от высоты камеры до нижней грани
    yy = 256
    point1 = [x1, yy]
    point2 = [x2, yy]
    point3 = [x2, horizontal_corners[1][x2]]
    point4 = [x1, horizontal_corners[1][x1]]

    height = y2_max - yy + 1
    src_points = np.array([point1, point2, point3, point4], dtype=np.float32)
    dst_points = np.array([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]], dtype=np.float32)

    # Получите матрицу преобразования
    perspective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)

    # Примените преобразование
    result_rectangular = cv2.warpPerspective(result, perspective_matrix, (width, height))

    for i in range(len(result_rectangular)):
        for k in range(width):
            result[i + yy][x1 + k] = result_rectangular[i][k]

In [76]:
width = 2
for j in range(x_min_2, x_max_2 + 1, width):
    x1 = j
    x2 = j + width - 1

    # от верхней грани до высоты камеры
    yy = 255
    point1 = [x1, horizontal_corners[0][x1]]
    point2 = [x2, horizontal_corners[0][x2]]
    point3 = [x2, yy]
    point4 = [x1, yy]

    height = yy - y1_max + 1
    src_points = np.array([point1, point2, point3, point4], dtype=np.float32)
    dst_points = np.array([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]], dtype=np.float32)

    # матрица преобразования
    perspective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)

    # преобразование
    result_rectangular = cv2.warpPerspective(result, perspective_matrix, (width, height))

    for i in range(len(result_rectangular)):
        for j in range(width):
            result[i + y1_max][x1 + j] = result_rectangular[i][j]



    # от высоты камеры до нижней грани
    yy = 256
    point1 = [x1, yy]
    point2 = [x2, yy]
    point3 = [x2, horizontal_corners[1][x2]]
    point4 = [x1, horizontal_corners[1][x1]]

    height = y2_max - yy + 1
    src_points = np.array([point1, point2, point3, point4], dtype=np.float32)
    dst_points = np.array([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]], dtype=np.float32)

    # Получите матрицу преобразования
    perspective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)

    # Примените преобразование
    result_rectangular = cv2.warpPerspective(result, perspective_matrix, (width, height))

    for i in range(len(result_rectangular)):
        for k in range(width):
            result[i + yy][x1 + k] = result_rectangular[i][k]

In [77]:
xxx = x_max_1 - x_min_1 + 1 + x_max_2 - x_min_2 + 1
yyy = y2_max - y1_max + 1

In [78]:
new_res = np.zeros((yyy, xxx, result.shape[2]), dtype=np.uint8)
new_res.shape

(84, 48, 3)

In [79]:
for i in range(x_min_1, x_max_1 + 1):
    for j in range(y1_max, y2_max + 1):
        new_res[j - y1_max][i - x_min_1] = result[j][i]

In [80]:
temp = x_max_1 - x_min_1 + 1
for i in range(x_min_2, x_max_2 + 1):
    for j in range(y1_max, y2_max + 1):
        new_res[j - y1_max][i - x_min_2 + temp] = result[j][i]

In [81]:
# cv2.imwrite("/content/6_wall.jpg", new_res)

In [82]:
result = new_res

In [83]:
depth_1_wall = list(depth_map[255][vertical_corners[10][0]:1024]) + list(depth_map[255][0:vertical_corners[0][0]+1])
depth_1_wall = np.array(depth_1_wall, dtype=np.float32)
depth_1_wall

array([5.7701054, 5.8151207, 5.812049 , 5.809199 , 5.80657  , 5.8041625,
       5.801975 , 5.800007 , 5.7982593, 5.79673  , 5.79542  , 5.794329 ,
       5.7934566, 5.792802 , 5.792366 , 5.7921476, 5.7921476, 5.792366 ,
       5.7928023, 5.7934566, 5.794329 , 5.7954206, 5.7967305, 5.7982593,
       5.8000073, 5.8019753, 5.8041625, 5.8065705, 5.8091993, 5.8120494,
       5.815121 , 5.8184156, 5.821933 , 5.825674 , 5.8296394, 5.8338304,
       5.8382473, 5.8428907, 5.847762 , 5.852862 , 5.8581915, 5.863752 ,
       5.869544 , 5.875569 , 5.881828 , 5.888322 , 5.895053 , 5.8855715],
      dtype=float32)

In [84]:
depth_1_wall[0] = depth_1_wall[1] + (depth_1_wall[1] - depth_1_wall[2])
depth_1_wall[0]

5.8181925

In [85]:
depth_1_wall[14:16] - min(depth_1_wall)

array([0.00021839, 0.        ], dtype=float32)

In [86]:
OA = depth_1_wall[0]
OB = depth_1_wall[-1]
OH = min(depth_1_wall)

AH = (OA**2 - OH**2)**0.5
BH = (OB**2 - OH**2)**0.5
(AH+BH) / (2.05-0.4)

0.9663019314943981

In [87]:
center_1_wall = 15
mera = (depth_1_wall[center_1_wall + 3] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5
mera

0.08708982050529868

In [88]:
(depth_1_wall[center_1_wall - 4] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5

0.15898505189333115

In [89]:
total_size = 0 + 4
ind_size_higher = [[center_1_wall, 4]]
ind = center_1_wall + 3
prev = (depth_1_wall[center_1_wall + 3] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5
while ind + 4 < len(depth_1_wall):
    cur = (depth_1_wall[ind + 4] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5
    next_size = np.around((cur - prev) / mera * 4, decimals = 0).astype(int)
    ind_size_higher.append([ind, next_size])
    total_size += next_size
    ind += 4
    prev = cur
if ind != depth_1_wall.shape[0] - 1:
    ind -= 4
    total_size -= ind_size_higher[-1][1]
    ind_size_higher.pop()
    step = depth_1_wall.shape[0] - 1 - ind
    cur =  ((depth_1_wall[ind + step] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5)
    prev = ((depth_1_wall[ind] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5)
    next_size = np.around((cur - prev) / mera * 4, decimals = 0).astype(int)
    ind_size_higher.append([ind, next_size])
    total_size += next_size
    ind += step
ind_size_higher.append([ind, 0])
high_size = total_size
#ind_size_higher

In [90]:
total_size = 4
ind = center_1_wall - 4
ind_size_lower = [[ind, 4]]
ind -= 4
prev = (depth_1_wall[center_1_wall + 3] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5
while ind >= 0:
    cur = ((depth_1_wall[ind] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5)
    next_size = np.around((cur - prev) / mera * 4, decimals = 0).astype(int)
    ind_size_lower.insert(0, [ind, next_size])
    total_size += next_size
    ind -= 4
    prev = cur
if ind != 0:
    ind += 4 * 2
    total_size -= ind_size_lower[0][1]
    ind_size_lower.pop(0)
    step = ind
    prev = ((depth_1_wall[ind] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5)
    cur = ((depth_1_wall[0] ** 2 - depth_1_wall[center_1_wall] ** 2) ** 0.5)
    next_size = np.around((cur - prev)/ mera * 4, decimals = 0).astype(int)
    ind_size_lower.insert(0, [0, next_size])
    total_size += next_size
    ind = 0
total_size += high_size
#ind_size_lower

In [91]:
ind_size = ind_size_lower + ind_size_higher
new_res = np.zeros((result.shape[0], total_size, result.shape[2]), dtype=np.uint8)
#new_res.shape

In [92]:
height = result.shape[0]
xx = 0
for j in range(len(ind_size) - 1):
    width = ind_size[j + 1][0] - ind_size[j][0]
    x1 = ind_size[j][0]
    x2 = x1 + width - 1

    yy = 255
    point1 = [x1, 0]
    point2 = [x2, 0]
    point3 = [x2, height - 1]
    point4 = [x1, height - 1]


    src_points = np.array([point1, point2, point3, point4], dtype=np.float32)
    dst_points = np.array([[0, 0], [ind_size[j][1] - 1, 0], [ind_size[j][1] - 1, height - 1], [0, height - 1]], dtype=np.float32)

    # Получите матрицу преобразования
    perspective_matrix = cv2.getPerspectiveTransform(src_points, dst_points)

    # Примените преобразование
    result_rectangular = cv2.warpPerspective(result, perspective_matrix, (ind_size[j][1], height))

    for i in range(len(result_rectangular)):
        for k in range(ind_size[j][1]):
            new_res[i][xx + k] = result_rectangular[i][k]
    xx += ind_size[j][1]

In [93]:
new_width = 165
new_height = 250

# Сжимаем изображение до нового размера
resized_image = cv2.resize(new_res, (new_width, new_height))
# получаем итоговое изображение с поправкой на соотношение сторон 1650x2500 мм
cv2.imwrite("/content/5_proj.jpg", resized_image)

True